In [12]:
import numpy as np
import pandas as pd
print(pd.__version__)

2.3.1


In [13]:
df=pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')
df.head().T


,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


columns are classified as  numerical and categorical based on their column values
Q1.Retail is the most frequent industry type of the leads

In [14]:
print(df.isnull().sum())
print(df.dtypes)
categorical_columns=list(df.columns[df.dtypes=='object'])
numerical_columns=list(df.columns[df.dtypes!='object'])


df[categorical_columns] = df[categorical_columns].fillna('NA')
df[numerical_columns] = df[numerical_columns].fillna(0)
df['industry'].mode()



lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64
lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object


0    retail
Name: industry, dtype: object

Q2.Correlation between columns

interaction_count and lead_score  0.011374
number_of_courses_viewed and lead_score  0.011529
number_of_courses_viewed and interaction_count  -0.050187
annual_income and interaction_count -0.015510

In [15]:
from sklearn.model_selection import train_test_split
df_fulltrain,df_test=train_test_split(df,test_size=0.2,random_state=42)
df_train,df_val=train_test_split(df_fulltrain,test_size=0.25,random_state=42)
sum_of_df = len(df_test)+len(df_train)+len(df_val)
print(sum_of_df,len(df))
assert len(df) == sum_of_df


df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
print(len(df_train),len(df_val),len(df_test))

y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test= df_test['converted'].values

del df_train['converted']
del df_val['converted']
del df_test['converted']
if 'converted' in numerical_columns:
    numerical_columns.remove('converted')

df_train[numerical_columns].corr()




1462 1462
876 293 293


,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.001875,-0.050187,0.011529
annual_income,0.001875,1.000000,-0.015510,0.048096
interaction_count,-0.050187,-0.015510,1.000000,0.011374
lead_score,0.011529,0.048096,0.011374,1.000000


Q3.Mutual information Ans: Lead_source

In [16]:

from sklearn.metrics import mutual_info_score

def mutual_info_score_series(series):
    return mutual_info_score(series,y_train)

mi=df_train[categorical_columns].apply(mutual_info_score_series)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

In [17]:
from IPython.display import display
for c in categorical_columns:
    display(df_fulltrain[c].unique())

array(['social_media', 'events', 'referral', 'paid_ads', 'organic_search',
       'NA'], dtype=object)

array(['manufacturing', 'retail', 'education', 'healthcare', 'finance',
       'NA', 'other', 'technology'], dtype=object)

array(['self_employed', 'student', 'NA', 'employed', 'unemployed'],
      dtype=object)

array(['australia', 'north_america', 'europe', 'asia', 'africa',
       'south_america', 'middle_east', 'NA'], dtype=object)

Q4. logistic regression  Ans 0.74

In [18]:

print(categorical_columns)
print(numerical_columns)
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient = 'records')
print(train_dict)
X_train=dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient = 'records')
X_val=dv.transform(val_dict)

test_dict = df_test[categorical_columns + numerical_columns].to_dict(orient = 'records')
X_test=dv.transform(test_dict)

['lead_source', 'industry', 'employment_status', 'location']
['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
[{'lead_source': 'paid_ads', 'industry': 'retail', 'employment_status': 'student', 'location': 'middle_east', 'number_of_courses_viewed': 0, 'annual_income': 58472.0, 'interaction_count': 5, 'lead_score': 0.03}, {'lead_source': 'organic_search', 'industry': 'manufacturing', 'employment_status': 'student', 'location': 'middle_east', 'number_of_courses_viewed': 3, 'annual_income': 71738.0, 'interaction_count': 6, 'lead_score': 0.77}, {'lead_source': 'paid_ads', 'industry': 'technology', 'employment_status': 'employed', 'location': 'north_america', 'number_of_courses_viewed': 3, 'annual_income': 81973.0, 'interaction_count': 2, 'lead_score': 0.59}, {'lead_source': 'NA', 'industry': 'technology', 'employment_status': 'employed', 'location': 'europe', 'number_of_courses_viewed': 1, 'annual_income': 74956.0, 'interaction_count': 3, 'lead_score': 0.34},

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
Accuracy_score = 0
def buildLogitic_Regression(c):
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    #print(X_train.shape)
    return model
 
model=buildLogitic_Regression(1)
model.fit(X_train,y_train)
y_pred= model.predict(X_val)
Accuracy_score =accuracy_score(y_val,y_pred)
print(round(Accuracy_score,3))

y_fulltrain = np.concatenate([y_train,y_val])
X_fulltrain_dict = train_dict+val_dict
X_fulltrain = dv.fit_transform(X_fulltrain_dict)
print(X_fulltrain.shape)
model.fit(X_fulltrain,y_fulltrain)
y_pred_n = model.predict(X_test)
print(round(accuracy_score(y_test,y_pred_n),2))

0.7
(1169, 31)
0.74


Q5.Feature elimination
Which of following feature has the smallest difference?

'industry'
'employment_status'
'lead_score'

Ans :industry

In [20]:

def calculate_diff_in_Accuracies(X_dataset,X_vald):
    
    model.fit(X_dataset,y_train)
    y_pred = model.predict(X_vald)
    score = accuracy_score(y_val,y_pred)
    display(Accuracy_score,score)
    return Accuracy_score-score



In [21]:
totalcolumns = categorical_columns+numerical_columns
differences = {}
for c in totalcolumns:
    df_new = df_train.copy()
    df_val_new =df_val.copy().drop(c,axis=1)
    df_new = df_new.drop(c,axis=1)
    #print(c)
    #print(df_new.T)
    xnew_dict = df_new.to_dict(orient ='records')
    xval_dic =df_val_new.to_dict(orient ='records')
    X_new_train =dv.fit_transform(xnew_dict)
    x_vald = dv.transform(xval_dic)
    differences[c]=round(calculate_diff_in_Accuracies(X_new_train,x_vald),3)
differences

0.6996587030716723

0.7030716723549488

0.6996587030716723

0.6996587030716723

0.6996587030716723

...

0.6996587030716723

0.5563139931740614

0.6996587030716723

0.7064846416382252

{'lead_source': -0.003,
 'industry': 0.0,
 'employment_status': 0.003,
 'location': -0.01,
 'number_of_courses_viewed': 0.143,
 'annual_income': -0.154,
 'interaction_count': 0.143,
 'lead_score': -0.007}

Q6.Regularizeed logistic regression

Ans : all reularizations have same accuracy 0.7
So smallest c =0.01 is chosen

In [22]:
for c in [0.01,0.1,1,10,100]:
    model=buildLogitic_Regression(c)
    model.fit(X_train,y_train)
    y_pred= model.predict(X_val)
    Accuracy_score =accuracy_score(y_val,y_pred)
    print(c,Accuracy_score)

0.01 0.6996587030716723
0.1 0.6996587030716723
1 0.6996587030716723
10 0.6996587030716723
100 0.6996587030716723
